In [ ]:
from __future__ import division
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
from PyQt5.QtWidgets import QApplication, QFileDialog
import matplotlib as mpl
mpl.rcParams['pdf.fonttype']=42
import fnmatch
%load_ext autoreload
%autoreload 2
%precision 4
mpl.rcParams['pdf.fonttype'] = 42 
import seaborn as sns
import scipy.signal as sg
sns.set_context("poster")
sns.set_style("ticks")
from collections import Counter
#import OpenEphys2 as OE
#import OpenEphys_V12 as OE
# 06/30/15 - version 1_01
import OpenEphys_V14 as OE
import pandas as pd
from pandas import DataFrame
import numpy as np
import scipy.stats as sstat
import glob
import os
import re
import mne
from scipy.signal import iirfilter, lfilter
import sys
import fnmatch
%matplotlib inline
mpl.rcParams['pdf.fonttype'] = 42 
mpl.rcParams['font.sans-serif']=['Arial', 'Helvetica','Bitstream Vera Sans', 'DejaVu Sans', 'Lucida Grande', 
                                 'Verdana', 'Geneva', 'Lucid', 'Avant Garde', 'sans-serif']  

# pal=sns.blend_palette(["black", "crimson"], 2)
sns.despine()
# current_palette = sns.color_palette("colorblind", 10)
# sns.set_palette(current_palette)

# for publication quality plots, not bar graphs, use this: 
def set_pub_plots(pal=sns.blend_palette([    "gray", "crimson", 'magenta', 'cyan', 'purple' ],5)):
    sns.set_style("white")
    sns.set_palette(pal)
    sns.set_context("poster", font_scale=1.5, rc={"lines.linewidth": 2.5, "axes.linewidth":2.5}) 
    sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8, 'figure.facecolor': 'white',})
    # optional, makes markers bigger, too, axes.linewidth doesn't seem to work
    plt.rcParams['axes.linewidth'] = 2.5

rc_pub={'font.size': 25, 'axes.labelsize': 25, 'legend.fontsize': 25.0, 
    'axes.titlesize': 25, 'xtick.labelsize': 25, 'ytick.labelsize': 25, 
    #'axes.color_cycle':pal, # image.cmap - rewritesd the default colormap
    'axes.linewidth':2.5, 'lines.linewidth': 2.5,
    'xtick.color': 'black', 'ytick.color': 'black', 'axes.edgecolor': 'black','axes.labelcolor':'black','text.color':'black'}
# to restore the defaults, call plt.rcdefaults() 

#set_pub_bargraphs()
set_pub_plots()



In [ ]:
#Ignore the details of these functions for now

def butter_bandpass(lowcut, highcut, fs, order=3):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = sg.butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=3):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sg.filtfilt(b, a, data)
    return y

In [ ]:
# genotype = {
#     137:3, 138:1, 473:1, 474:3, 470:3, 471:1, 472:1, 231:2, 232:2, 352:1, 52:1, 53:1, 54:1, 55:2, 56:2, 57:2,
#     173:3, 174:3, 175:3, 283:3, 285:3, 286:2, 287:2
# }
genotype = {
    '137':'wt', '138':'ko', '473':'ko', '474':'wt', '470':'wt', '471':'ko', '472':'ko', '231':'het', '232':'het', 
    '173':'wt', '174':'wt', '175':'wt', '283':'wt', '285':'wt', '286':'het', '287':'het', '383':'wt', '273':'wt', 
    '420':'het', '421':'ko', '379':'het', '380':'het', '381':'ko', '382':'wt'
}
sert_groups = { '002': 'ko', '003': 'ko', '005': 'wt', '006': 'het', '007':'het', '008': 'het', '009':'ko', '010': 'ko', 
    '367': 'wt', '369': 'ko', '377': 'het', '378': 'ko', '380': 'ko', '488': 'wt', '489': 'ko', 
           '930': 'ko', '931': 'het', '936': 'het', '937': 'ko', '942': 'wt', '944': 'het', '958': 'ko', '959': 'het'
}

In [ ]:
import oe_map_functions_v1 as oem

## Generate pkl files 

In [ ]:
exclude_keyphrases=['Bad Reports','bad', 'MSF', 'phy']
directory = r"U:\Data\pak6\figures\LFP\probe_64DA\SERT KO\WT"
matches = [os.path.join(dirpath, f)
            for dirpath, dirnames, files in os.walk(directory)
            for f in files 
            if (f.endswith('.h5') and not any(filter(lambda bad: bad in f, exclude_keyphrases)))]
matches = [f for f in matches if 'bad' not in f and '201' in f]
matches[:5]

In [ ]:
for fname in matches[:]: 
    et = fname.split('\\')[-2].split()[0]
    training =  fname.split('\\')[-1].split()[1]
    rec = fname.split('\\')[-1].split()[-1].split('_')[1]
    stim1 =  fname.split('\\')[-1].split('_')[3][:3]
#     training = fname.split('\\')[-2].split()[-1]
#     if 'pre' in training or '1' in training:
#         training = 'pre'
#     else:
#         training = 'post'

    print et, training, rec, stim1
    

In [ ]:
matches_2 = [f for f in matches if 'bad' not in f and 'opto' not in f]

## Stim1-2 data

In [ ]:
trial_length = 4000

ls = []

for fname in matches[:]:
    print fname
#     tmp = tmp.groupby(np.arange((tmp.shape[0]))//30, axis = 0).mean() #downsample to 1kHz

    et = fname.split('\\')[-2].split()[0]
    training =  fname.split('\\')[-1].split()[1]
    rec = fname.split('\\')[-1].split()[-1].split('_')[1]
    stim1 =  fname.split('\\')[-1].split('_')[3][:3]
    
    if 'pre' in training or '1' in training:
        training = 'pre'
    else:
        training = 'post'
    
    
    try:
        tmp =  pd.read_hdf(fname, key = 'raw') 
#         tmp =  pd.read_hdf(fname, key = 'layer4') 
    except:
        print fname
        continue
   
    trials =  int(tmp.shape[1]/trial_length)
    
    times = np.linspace(0, trial_length/1e3, trial_length)
    df3_array=np.reshape(tmp.values,(np.shape(tmp)[0],trials, -1))
    df3_avg=np.mean(df3_array,1)
    df3_avg = pd.DataFrame(df3_avg).T        
    df3_avg.loc[:,'times'] = times
    negativity_ch_idx=df3_avg[(df3_avg['times'] >= 1) & (df3_avg['times'] < 1.15)].ix[:,11:16].min().idxmin()
#     negativity_ch_idx = negativity_ch_idx - 4
    print negativity_ch_idx

    # plotting the spectra and FFR for the maximum negativity channel, the first trial
    
    ddf = df3_avg.iloc[:, negativity_ch_idx]

    tmp_df = pd.DataFrame({'et': et, 'lfp':ddf, 'stim1':stim1,  'training': training,  'ch': negativity_ch_idx,
                           'fname':fname, 'times':times, 'layer': 'l4', 'rec':rec
                          })
    ls.append(tmp_df)


df_sert_l4 = pd.concat(ls)

In [ ]:
df_sert_l4.head()

In [ ]:
df_sert_l5['group'] = 'wt'

In [ ]:
for fn in df_serta.fname.unique():
    plt.plot(df_serta[df_serta.fname == fn].lfp)
    plt.show()

In [ ]:
df_sert_l4.to_pickle('sert_ko_stim1_training_l4-lfp,pkl')

In [ ]:
novel_oddball = np.array([3, 3, 3, 3, 3, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 
3, 9, 3, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3, 
3, 3, 3, 3, 3, 9, 3, 3, 3, 9, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3, 3, 
3, 3, 3, 3, 3, 3, 3, 9, 3, 3, 3, 9, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 
3, 9, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3, 9, 3, 3, 
3, 3, 9, 3, 3, 3, 3, 9, 3, 3, 9, 3, 3, 9, 3, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3, 3, 3, 
3, 9, 3, 3, 3, 3, 3, 9, 3, 3, 9, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 9, 3, 3, 3, 
3, 3, 3, 3, 9, 3, 3, 3, 3, 3, 3, 9, 3, 3, 3])

standard_control = np.array([4, 5, 3, 6, 5, 7, 0, 3, 7, 6, 7, 1, 2, 6, 7, 0, 5, 0, 7, 0, 7, 3, 2, 1, 7, 0, 6, 1, 3, 1, 4, 6, 6, 5, 1, 4, 4, 5, 2, 3, 0, 5, 2, 3, 4, 2, 6, 5, 6, 6, 3, 7, 0, 1, 4, 6, 4, 0, 6, 6, 7, 4, 3, 6, 1, 1, 5, 4, 0, 4, 0, 2, 6, 2, 6, 7, 7, 7, 3, 0, 1, 7, 6, 1, 3, 2, 3, 5, 7, 0, 1, 5, 2, 3, 4, 2, 4, 6, 0, 4, 3, 2, 4, 1, 2, 0, 5, 1, 5, 3, 4, 6, 3, 5, 2, 2, 7, 1, 6, 0, 0, 5, 4, 0, 1, 5, 0, 3, 7, 5, 7, 4, 6, 2, 0, 0, 1, 1, 0, 5, 3, 2, 4, 7, 0, 7, 5, 2, 7, 1, 2, 7, 3, 0, 2, 3, 6, 2, 7, 3, 5, 3, 4, 5, 3, 7, 4, 2, 6, 6, 2, 1, 3, 1, 4, 5, 0, 2, 2, 6, 1, 4, 5, 1, 6, 3, 0, 5, 3, 2, 5, 4, 1, 7, 4, 4, 1, 5, 7, 1])

sf_tuning = np.array([2, 0, 0, 4, 3, 3, 0, 4, 4, 3, 2, 5, 3, 2, 0, 4, 1, 0, 5, 2, 0, 
1, 0, 1, 3, 5, 2, 5, 1, 2, 0, 5, 2, 3, 5, 1, 0, 4, 3, 2, 5, 5, 3, 5, 
2, 0, 3, 3, 0, 3, 4, 5, 4, 1, 4, 0, 1, 5, 4, 1, 5, 3, 3, 5, 3, 3, 2, 
3, 2, 1, 1, 5, 1, 4, 1, 2, 3, 2, 4, 2, 1, 0, 5, 5, 2, 2, 4, 1, 4, 1, 
3, 1, 0, 4, 4, 4, 4, 0, 5, 4, 4, 0, 3, 5, 5, 2, 1, 3, 4, 1, 5, 0, 2, 
2, 0, 0, 0, 1, 2, 1])

loc_omission = np.array([3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 
0, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 
3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 
3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 0, 
3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 
3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 
3, 0, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 0, 3, 
3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 0])

dir_seq = np.array([10, 7, 3, 2, 4, 8, 9, 5, 7, 3, 4, 8, 3, 2, 1, 8, 0, 4, 9, 11, 
10, 9, 1, 11, 4, 0, 7, 1, 2, 8, 2, 9, 11, 9, 6, 5, 10, 4, 9, 0, 7, 11, 9, 
5, 9, 10, 11, 6, 8, 9, 5, 4, 2, 8, 11, 2, 10, 3, 5, 1, 7, 0, 4, 9, 1, 5, 
11, 3, 5, 10, 1, 2, 9, 6, 2, 2, 11, 5, 10, 7, 3, 7, 4, 6, 8, 4, 1, 8, 0, 
11, 0, 6, 2, 11, 1, 10, 3, 8, 3, 1, 2, 10, 5, 3, 11, 1, 7, 3, 4, 7, 8, 4, 6, 
7, 11, 7, 0, 8, 6, 10, 4, 5, 7, 2, 10, 3, 5, 9, 8, 6, 3, 2, 0, 11, 0, 6, 10, 
0, 7, 4, 5, 0, 10, 6, 8, 10, 3, 11, 9, 0, 5, 1, 3, 7, 0, 6, 9, 1, 6, 10, 5, 
6, 11, 7, 0, 5, 1, 4, 1, 6, 8, 2, 9, 2, 8, 3, 0, 4, 6, 1])

In [ ]:
# groups = {'ET#304': 1, 'ET#271': 1, 'ET#309': 2, 'ET#270': 2, 'ET#313': 1, 'ET#307': 2}
# groups = {'ET#970': 'b', 'ET#971': 'b', 'ET#972': 'b', 'ET#973': 'a', 'ET#974': 'a', 'ET#975': 'a', 'ET#758': 'a',
#          'ET#759': 'a'}
groups = {'479': 'wt', '480': 'fx', '481':'fx', '482': 'fx', '483': 'wt', '484': 'wt', '485':'wt', '486':'fx', 
          '013': 'wt', '014': 'fx', '016': 'fx', '017': 'wt', '018': 'fx', '019': 'wt', '020': 'wt', '022': 'fx',
         '023': 'wt', '024': 'fx', '025': 'fx', '026': 'fx', '027': 'fx', '028': 'fx', '029': 'wt', '030': 'fx', '031': 'fx', '032': 'fx', 
          '033': 'fx' , '034': 'fx', '035': 'wt', 'W00': 'wt', 'W01': 'wt',
 'W02': 'wt', 'W03': 'wt', 'W04': 'wt', 'W05':'wt'}


## Generate pkl for SF-vmmn data

In [ ]:
ls = []

for fname in matches[:]:
    stim_type = ''
    et = fname.split('\\')[-1].split()[2][3:]
    side = 'left-1'
    paradigm = fname.split('\\')[-1].split()[-1].split('_')[2]
    
    if 'bad' in fname or 'sf' not in fname:
        continue
    print fname

    if 'nov' in paradigm:
        trial_type = novel_oddball
        idx_odd = np.where(trial_type==9)[0]
        pre_odd = idx_odd-1
        print 'novel oddball'
    elif 'omis' in paradigm:
        trial_type = loc_omission
        idx_odd = np.where(trial_type==0)[0]
        pre_odd = idx_odd-1
        print 'loc_omission'
    elif 'control' in paradigm:
        trial_type = standard_control
        print 'std control'
    elif 'tuning' in paradigm:
        trial_type = sf_tuning
        print 'tuning'
    else:
        print 'no paradigm'
        continue
        
    try:
        tmp =  pd.read_hdf(fname, key = 'raw') 
#         trial_length = pd.read_hdf(fname, key = 'trial_duration') 
    except:
        continue
    trial_length = 1000
    trials =  int(tmp.shape[1]/trial_length)
    
    times = np.linspace(0, trial_length/1e3, trial_length)
    df3_array=np.reshape(tmp.values,(np.shape(tmp)[0],trials, -1))
    df3_avg=np.mean(df3_array,1)
    df3_avg = pd.DataFrame(df3_avg).T        
    df3_avg['times'] = times
    negativity_ch_idx=df3_avg[(df3_avg['times']>=0.35) & (df3_avg['times']< 0.5)].ix[:,8:16].min().idxmin()
#     negativity_ch_idx = negativity_ch_idx - 4
    print negativity_ch_idx

    # plotting the spectra and FFR for the maximum negativity channel, the first trial
    
    ddf = tmp.values[negativity_ch_idx,:]
    
    ddf2=np.reshape(ddf,(trials, -1))
    print ddf2.shape
    ddf2 = pd.DataFrame(ddf2)
    
    if 'control' in paradigm or 'tuning' in paradigm:
        for j in np.unique(trial_type):
            stim_type = 'ctr'
            lfp = ddf2.iloc[np.where( trial_type == j)[0]].mean()
            tmp_df = pd.DataFrame({'et': et, 'lfp':lfp, 'paradigm': paradigm,  'stim1': stim_type + str(j), 
                        'layer': 'l2/3',
                           'side':side,    'fname':fname, 'times':times, 'ch_idx': negativity_ch_idx })
            ls.append(tmp_df)
    else:
        for j in np.unique(trial_type):
            if j == 3:
                stim_type = 'std'
                lfp = ddf2.iloc[pre_odd].mean()
            elif j == 9 or j == 0:
                stim_type = 'dev'
                lfp = ddf2.iloc[idx_odd].mean()
            else:
                continue
    #         if j==3 or j ==9:
    #             plt.plot(lfp)
    #             plt.show()
            tmp_df = pd.DataFrame({'et': et, 'lfp':lfp, 'paradigm': paradigm,  'stim1': stim_type + str(j), 
                                    'layer': 'l2/3',
                   'side':side,  'fname':fname, 'times':times, 'ch_idx': negativity_ch_idx })
            ls.append(tmp_df)
    
mmn_df4 = pd.concat(ls)
    

In [ ]:
mmn_df4['group'] = mmn_df4.et.map(groups)

In [ ]:
sorted(mmn_df4.group.unique())

In [ ]:
mmn_df4.head()

## pkl file for standard ori vmmn

In [ ]:
ls = []

oddball = np.array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 
3, 3, 3, 3, 3, 3,
 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
 3, 3, 3, 3, 3, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3,
 3, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
 3, 3, 9, 3, 3, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
 9, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3, 9, 3, 3,
 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 9, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3, 3,
 3, 3, 9, 3, 3, 3, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3, 3, 3, 9, 3, 3, 3,
 3, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3, 9, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3,
 3, 3, 3, 3, 9, 3, 3, 3, 3, 3, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3, 3, 3,
 9, 3, 3, 3, 9, 3, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3,
 3, 3, 3, 9, 3, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3, 3, 3, 9, 3, 3, 3, 3,
 3, 3, 9, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3, 9, 3,
 3, 3, 3, 3, 3, 3, 3, 9, 3, 3, 3, 3, 3, 9])

control = np.array([8, 12, 4, 9, 4, 10, 12, 8, 2, 10, 4, 9, 9, 4, 3, 2, 10, 2, 4, 10, 8, 4, 10, 10, 12, 3, 10, 2, 3, 10, 9, 2, 8, 6, 2, 10, 12, 9, 9, 2, 6, 4, 10, 12, 12, 12, 3, 6, 4, 4, 12, 6, 3, 6, 4, 10, 3, 3, 6, 3, 8, 3, 12, 6, 9, 12, 10, 3, 8, 8, 12, 6, 9, 9, 3, 10, 2, 6, 10, 4, 12, 9, 2, 10, 9, 4, 3, 2, 9, 12, 2, 8, 8, 8, 12, 8, 9, 2, 8, 6, 10, 8, 10, 2, 8, 9, 8, 9, 9, 3, 4, 2, 8, 4, 2, 3, 4, 8, 12, 4, 6, 10, 12, 9, 8, 8, 3, 2, 6, 6, 3, 10, 4, 10, 2, 9, 10, 6, 3, 10, 8, 12, 9, 3, 8, 4, 2, 8, 4, 8, 12, 4, 6, 12, 10, 10, 9, 4, 12, 10, 3, 6, 2, 9, 4, 12, 2, 12, 3, 2, 6, 9, 6, 8, 2, 4, 3, 2, 6, 9, 2, 8, 12, 6, 10, 6, 9, 6, 6, 8, 2, 4, 9, 3, 9, 6, 12, 10, 2, 12, 3, 8, 6, 3, 3, 12, 3, 12, 6, 9, 6, 12, 3, 8, 4, 9, 12, 2, 2, 2, 9, 10, 12, 8, 4, 4, 6, 10, 10, 6, 9, 8, 3, 3, 4, 4, 3, 4, 2, 6])

control_old = np.array([10,  2, 10,  8,  9,  9,  2,  2, 10,  4,  3,  2, 10,  4,  9,  3,  3,
        4,  9,  2,  8,  9,  3, 10,  2,  4,  8, 10,  9,  9,  3, 10,  2,  2,
        2,  4,  2,  3,  8,  8,  3,  3,  4,  2, 10, 10, 10,  3,  3,  8,  8,
        4,  4,  9,  2,  9, 10,  8, 10,  8, 10,  8,  3,  2,  3,  9,  4,  3,
        4,  3,  2,  2, 10,  4,  8,  8,  2,  8,  9,  9,  4,  2, 10,  3,  9,
        3,  2, 10,  3,  2,  3,  8,  4,  9,  3, 10,  9,  4, 10,  8,  2, 10,
        8, 10,  4,  4,  4,  2, 10,  9,  4, 10,  9,  9,  9,  4,  3, 10,  3,
       10, 10,  9,  8,  4,  4,  2,  4,  3,  2,  9, 10,  2,  9,  3, 10,  8,
        8,  4,  4,  3,  2,  3,  4, 10,  9, 10,  2,  8, 10,  9,  8, 10,  2,
        4, 10,  2,  8,  9,  2,  8,  3, 10,  8, 10,  4,  4,  8,  4,  8,  9,
        3,  9,  2,  3,  4,  3,  4,  3,  8,  8,  3,  9,  3,  2,  9,  4, 10,
       10,  9,  4,  2,  4,  3,  2,  8,  3,  8,  8, 10,  9,  2,  3,  9,  4,
        4, 10,  4,  4,  2,  4,  8,  9,  8, 10,  8,  2,  8,  3,  2,  2,  2,
        8,  3, 10,  3,  8,  3,  2, 10,  9,  3,  9, 10,  2,  3,  8,  8,  9,
        9,  9,  8,  9,  9,  9,  9,  4,  4,  2,  4,  2,  4,  9, 10,  4,  3,
        3,  8,  8, 10,  8,  8,  8,  3,  9,  9, 10,  8,  3, 10, 10,  2,  9,
        4,  8,  8,  2,  3,  9,  2,  9, 10,  3, 10,  4,  3,  3,  4, 10,  2,
        4,  9,  4,  8,  9,  2,  8,  2,  4,  3,  2])

for fname in matches:
    paradigm = fname.split('\\')[-1].split()[-1].split('_')[2]
    exp = fname.split('\\')[-1].split()[-1].split('_')[1]
    et = fname.split('\\')[-1].split()[0]
    rec = fname.split('\\')[-1].split()[-1].split('_')[0]
    
#     if 'oddball' not in paradigm:
#         continue
    print fname

    if 'odd' in paradigm:
        trial_type = oddball
        idx_odd = np.where(trial_type==9)[0]
        pre_odd = idx_odd-1
        print 'ori oddball'
    elif 'control' in paradigm:
        trial_type = control
        print 'control'
    else:
        print 'no paradigm'
        continue
        
    try:
        tmp =  pd.read_hdf(fname, key = 'raw') 
#         trial_length = pd.read_hdf(fname, key = 'trial_duration') 
    except:
        continue
    trial_length = 1000
    trials =  int(tmp.shape[1]/trial_length)
    if trials != 300:
        continue
    else:
        print 'good'
    times = np.linspace(0, trial_length/1e3, trial_length)
    df3_array=np.reshape(tmp.values,(np.shape(tmp)[0],trials, -1))
    df3_avg=np.mean(df3_array,1)
    df3_avg = pd.DataFrame(df3_avg).T        
    df3_avg['times'] = times
    negativity_ch_idx=df3_avg[(df3_avg['times']>=0.35) & (df3_avg['times']< 0.45)].ix[:,8:16].min().idxmin()
#     negativity_ch_idx = negativity_ch_idx - 4
    print negativity_ch_idx

    # plotting the spectra and FFR for the maximum negativity channel, the first trial
    
    ddf = tmp.values[negativity_ch_idx,:]
    
    ddf2=np.reshape(ddf,(trials, -1))
    print ddf2.shape
    ddf2 = pd.DataFrame(ddf2)
    
    if 'control' in paradigm:
        for j in np.unique(trial_type):
            lfp = ddf2.iloc[np.where( trial_type == j)[0]].mean()
            tmp_df = pd.DataFrame({'et': et, 'lfp':lfp, 'paradigm': paradigm,  'stim1': 'std' + str(j), 
                           'rec':rec,     'fname':fname, 'times':times })
            ls.append(tmp_df)
    else:
        for j in np.unique(trial_type):
            if j == 3 or j == 1:
                lfp = ddf2.iloc[pre_odd].mean()
            elif j == 9 or j == 2:
                lfp = ddf2.iloc[idx_odd].mean()
            else:
                continue
    #         if j==3 or j ==9:
    #             plt.plot(lfp)
    #             plt.show()
            tmp_df = pd.DataFrame({'et': et, 'lfp':lfp, 'paradigm': paradigm,  'stim1':'odd'+ str(j), 'rec':rec, 
                   'fname':fname, 'times':times })
            ls.append(tmp_df)
    
mmn_ori = pd.concat(ls)
    

In [ ]:
mmn_ori.head()

In [ ]:
df_hdac3.et.unique().size

##  .pkl for trial-based LFP analysis

In [ ]:
ls = []
trial_length = 1000
trials = sf_tuning.size
times = np.linspace(0, trial_length/1e3, trial_length)

for fname in matches[:]: 
    df2 = 0
    et = fname.split('\\')[-2].split()[0]
    side =  fname.split('\\')[-1].split()[-1].split('_')[0]
    rec = et+side
    
    try:
        tmp =  pd.read_hdf(fname, key = 'raw') 
#         trial_length = pd.read_hdf(fname, key = 'trial_duration') 
    except:
        continue
    
    df3_array = np.reshape(tmp.values,(np.shape(tmp)[0],trials, -1))
    df3_avg = np.mean(df3_array,1)
    df3_avg = pd.DataFrame(df3_avg).T        
    df3_avg.loc[:,'times'] = times
    negativity_ch_idx = df3_avg[(df3_avg['times']>=0.35) & (df3_avg['times']< 0.5)].ix[:,8:16].min().idxmin()
    # convert to long form df
    df = pd.DataFrame(df3_array[negativity_ch_idx]).T
    df.columns = sf_tuning
    df.loc[:, 'times'] = times
    df2 = pd.melt(df, id_vars = 'times', var_name = 'stim1', value_name = 'lfp')
    df2 = df2.round(4)
    df2.loc[:, 'et'], df2.loc[:, 'side'], df2.loc[:, 'rec'], df2.loc[:, 'fname'] = et, side, rec, fname
    ls.append(df2)


In [ ]:
master_sf = pd.concat(ls)
master_sf.head()

In [ ]:
master_df.to_pickle('master_fx_mmn_sf-tuning_bytrial.pkl')

In [ ]:
master_df = pd.concat([tmp, tmp2, tmp3])

In [ ]:
master_df.head()

## Generate CSD pkl file

### Stim1 training

In [ ]:
times.size

In [ ]:
ls2 = []
times = np.linspace(0, 1.0, 4000)
trial_length = 4000
f, ax = plt.subplots(2, figsize = (1,1))
for fname in matches[:]:
    if 'bad' in fname:
        continue

    tmp_df = 0
    
    et = fname.split('\\')[-2].split()[0]
    training = fname.split('\\')[-2].split()[-1]
    if 'pre' in training or '1' in training:
        training = 'pre'
    else:
        training = 'post'
    stim1 =  fname.split('\\')[-1].split('_')[3][:3]
    
    ls = []

    try:
        tmp =  pd.read_hdf(fname, key = 'raw') 
    except:
        continue
    trials =  int(tmp.shape[1]/trial_length)


    csd_tmp = np.split(tmp, int(trials), axis = 1)
    csd_input = np.dstack(csd_tmp)
    

    csd_tmp = csd_input.mean(axis = 2)

    csd_tmp = pd.DataFrame(csd_tmp)
    csd_tmp = csd_tmp - csd_tmp.median(axis = 0)

    csd = oem.df_CSD_analysis( np.array(csd_tmp), ax, 
                      Channel_Number=np.shape(csd_tmp)[0], show_plot=False)
    tmp_df = pd.DataFrame(csd).stack().reset_index()
    tmp_df.columns = ['csd_step', 'samples', 'csd']
    tmp_df['et'] = et
#     tmp_df['paradigm'] = paradigm
    tmp_df['stim1'] = stim1
    tmp_df['fname'] = fname
    tmp_df['times'] = tmp_df['samples']/1000
    tmp_df['training'] = training 
    
    ls.append(tmp_df)

    
    tmp_df2 = pd.concat(ls)
    ls2.append(tmp_df2)
    
csd_df2 = pd.concat(ls2)
    

In [ ]:
csd_df2.et.unique()

In [ ]:
# adjust time: stim1 at 1s. 380 pre, 377, 378 post
# stim1 at 0.5s: 009, 010 pre, rest at 1s
v = '378'
csd_df2.loc[((csd_df2.et == v) & (csd_df2.training == 'post')), 'times'] = csd_df2.loc[((csd_df2.et == v) 
                                                    & (csd_df2.training == 'post')), 'times'] - 0.50
csd_df2 = csd_df2.round({'times': 3})

In [ ]:
# for probe 64DB, adjust times
for i,v in enumerate((sorted(csd_df2[csd_df2.et.str.startswith('0')].et.unique()))[:]):
    csd_df2.loc[((csd_df2.et == v) & (csd_df2.training == 'post')), 'times'] = csd_df2.loc[((csd_df2.et == v) 
                                                               & (csd_df2.training == 'post')), 'times'] - 0.50
    csd_df2 = csd_df2.round({'times': 3})

### Oddball

In [ ]:
ls2 = []
times = np.linspace(0, 1.0, 1000)
f, ax = plt.subplots(2, figsize = (1,1))
for fname in matches:
    if 'bad' in fname:
        continue
    elif 'sf' in fname:
        print fname
    else:
        continue
    tmp_df = 0
    target_trials = ''
    stim_type = ''
    et = fname.split('\\')[-1].split()[0][1:]
    side = fname.split('\\')[-1].split()[1].split('_')[0]
    paradigm = fname.split('\\')[-1].split()[1].split('_')[2]
    
    ls = []

    if 'nov' in paradigm:
        trial_type = novel_oddball
        idx_odd = np.where(trial_type==9)[0]
        pre_odd = idx_odd-1
        print 'novel oddball'
    elif 'omis' in paradigm:
        trial_type = loc_omission
        idx_odd = np.where(trial_type==0)[0]
        pre_odd = idx_odd-1
        print 'loc_omission'
    elif 'control' in paradigm:
        trial_type = standard_control
        print 'std control'
    elif 'tuning' in paradigm:
        trial_type = sf_tuning
        print 'tuning'
    else:
        print 'no paradigm'
        continue
        
    try:
        tmp =  pd.read_hdf(fname, key = 'raw') 
    except:
        continue
    trials =  int(tmp.shape[1]/1000)


    csd_tmp = np.split(tmp, int(trials), axis = 1)
    csd_input = np.dstack(csd_tmp)
    
    if 'control' in paradigm or 'tuning' in paradigm:
        for j in np.unique(trial_type):
            stim_type = 'ctr'
            target_trials = np.where( trial_type == j)[0]
            csd_tmp = csd_input[:,:, target_trials].mean(axis = 2)
          
            csd_tmp = pd.DataFrame(csd_tmp)
            csd_tmp = csd_tmp - csd_tmp.median(axis = 0)

            csd = oem.df_CSD_analysis( np.array(csd_tmp), ax, 
                              Channel_Number=np.shape(csd_tmp)[0], show_plot=False)
            tmp_df = pd.DataFrame(csd).stack().reset_index()
            tmp_df.columns = ['csd_step', 'samples', 'csd']
            tmp_df['et'] = et
            tmp_df['paradigm'] = paradigm
            tmp_df['stim1'] = stim_type + str(j)
            tmp_df['fname'] = fname
            tmp_df['times'] = tmp_df['samples']/1000
            tmp_df['side'] = side 
            ls.append(tmp_df)
    else:
        for j in np.unique(trial_type):
            if j == 3:
                stim_type = 'std'
                csd_tmp = csd_input[:,:, pre_odd].mean(axis = 2)
            elif j == 9 or j == 0:
                stim_type = 'dev'
                csd_tmp = csd_input[:,:, idx_odd].mean(axis = 2)
            else:
                continue
            csd_tmp = pd.DataFrame(csd_tmp)
            csd_tmp = csd_tmp - csd_tmp.median(axis = 0)

            csd = oem.df_CSD_analysis( np.array(csd_tmp), ax, 
                              Channel_Number=np.shape(csd_tmp)[0], show_plot=False)
            tmp_df = pd.DataFrame(csd).stack().reset_index()
            tmp_df.columns = ['csd_step', 'samples', 'csd']
            tmp_df['et'] = et
            tmp_df['paradigm'] = paradigm
            tmp_df['stim1'] = stim_type + str(j)
            tmp_df['fname'] = fname
            tmp_df['times'] = tmp_df['samples']/1000
            tmp_df['side'] = side 
            ls.append(tmp_df)
    
    tmp_df2 = pd.concat(ls)
    ls2.append(tmp_df2)
    
csd_df2 = pd.concat(ls2)
    

In [ ]:
csd_df2['group'] = csd_df2.et.map(sert_groups)
csd_df2.head()

In [ ]:
csd_df.paradigm.unique()

In [ ]:
cond = 'stim1'
data = csd_df
_data = data[(data.paradigm.str.contains('odd'))
              | (data.stim1 == 'ctr2') 
               ] 
_data = _data[ (_data.times > 0.25)  & (_data.times < 0.75)]

n = sorted(_data[cond].unique())[::-1]

f,ax = plt.subplots(1, len(n), figsize = (12,6), sharey= True, sharex=True,facecolor = 'w')
cbar_ax = f.add_axes([.97, .3, .03, .4])
formatter = tkr.ScalarFormatter(useMathText=True)
formatter.set_scientific(True)
formatter.set_powerlimits((-3, 3))

for idx, val in enumerate(n):
    hm_input = _data[_data[cond] == val].groupby(['csd_step', 
            'times']).mean().reset_index().pivot('csd_step', 'times', 'csd').values
    sns.heatmap(hm_input, cmap = 'jet',  annot=False, xticklabels = 250,  yticklabels = 5,
                ax = ax[idx], cbar=idx == 0,
                vmin = -3.0e6, vmax = 3.0e6, robust = True,   
                cbar_ax = None if idx else cbar_ax,  cbar_kws={"format": formatter}
               )
    ax[idx].invert_yaxis()
    ax[idx].set_title(val)
    
# plt.savefig("hm-csd-cbar.png", transparent=True)

In [ ]:
sorted(_data[(_data.times == 0.4) & (_data.csd < -6e6)].csd_step.unique())

In [ ]:
tmp = _data[(_data.csd_step > 6) 
            & (_data.csd_step<30) 
            & (_data.stim1 == 'ctr2')
           ]
tmp.groupby('times').mean().csd.plot()

In [ ]:
df_serta.stim1.unique()

In [ ]:
df_serta.groupby(['group', 'et', 'paradigm']).fname.unique()

In [ ]:
path = r"U:\Data_Analysis\pak6\Analysis of LFP\fx_mmn_paper-lfp_master.pkl"
lfp_df = pd.read_pickle(path)

path = r"U:\Data_Analysis\pak6\Analysis of LFP\FX mmn paper\fx_mmn_paper-csd_master.pkl"
csd_df = pd.read_pickle(path)

In [ ]:
tmp = csd_df[csd_df.stim1 == 'dev9']
for i in sorted(tmp.csd_step.unique()[:]):
    plt.plot((tmp[tmp.csd_step == i].groupby('times').mean().csd))
    plt.show()
sns.despine()

In [ ]:
csd_df.loc[:, 'layer'] = 'un'
csd_df.loc[csd_df['csd_step'] < 11, 'layer'] = 'l6'
csd_df.loc[(csd_df['csd_step'] >= 11) & (csd_df['csd_step'] < 22), 'layer'] = 'l5'
csd_df.loc[(csd_df['csd_step'] >= 22) & (csd_df['csd_step'] <= 30), 'layer'] = 'l4'
# csd_df.loc[(csd_df['csd_step'] >= 24) & (csd_df['csd_step'] <= 32), 'layer'] = 'l5'
csd_df.loc[csd_df['csd_step'] > 30, 'layer'] = 'l2/3'

In [ ]:
path = r"U:\Data_Analysis\pak6\Analysis of LFP\SERT KO project\sert_l4_lfp_master.pkl"
sert_df = pd.read_pickle(path)

In [ ]:
lfp_df.paradigm.unique()

## Analysis of Tuning

In [ ]:
ls2 = []
times = np.linspace(0, 1.0, 1000)

groups = {'ET#304': 1, 'ET#271': 1, 'ET#309': 2, 'ET#270': 2, 'ET#313': 1, 'ET#307': 2}
sf_seq = [2, 0, 0, 4, 3, 3, 0, 4, 4, 3, 2, 5, 3, 2, 0, 4, 1, 0, 5, 2, 0, 
1, 0, 1, 3, 5, 2, 5, 1, 2, 0, 5, 2, 3, 5, 1, 0, 4, 3, 2, 5, 5, 3, 5, 
2, 0, 3, 3, 0, 3, 4, 5, 4, 1, 4, 0, 1, 5, 4, 1, 5, 3, 3, 5, 3, 3, 2, 
3, 2, 1, 1, 5, 1, 4, 1, 2, 3, 2, 4, 2, 1, 0, 5, 5, 2, 2, 4, 1, 4, 1, 
3, 1, 0, 4, 4, 4, 4, 0, 5, 4, 4, 0, 3, 5, 5, 2, 1, 3, 4, 1, 5, 0, 2, 
2, 0, 0, 0, 1, 2, 1]

contrast_seq = [1, 0.25, 0.125, 1, 0.0625, 1, 1, 0.5, 0.0625, 0.125, 0.5, 
0.125, 0.5, 0.5, 0.125, 0.125, 0.5, 0.25, 0.5, 0.0625, 1, 0.0625, 0.5, 1, 
1, 0.125, 1, 0.125, 0.125, 0.25, 0.0625, 0.0625, 0.125, 0.5, 0.125, 0.5, 
0.0625, 1, 1, 0.25, 0.25, 1, 1, 0.25, 0.125, 1, 0.25, 0.5, 0.0625, 0.5, 
0.125, 0.0625, 0.25, 0.0625, 0.25, 0.5, 0.25, 0.5, 0.125, 0.125, 1, 1, 
0.5, 0.5, 0.125, 1, 0.25, 0.0625, 0.25, 0.25, 1, 1, 0.25, 0.5, 0.5, 0.125, 
1, 0.125, 0.0625, 0.125, 0.25, 0.0625, 0.25, 0.0625, 0.0625, 0.0625, 0.5, 
0.125, 0.25, 0.5, 0.0625, 0.25, 0.5, 0.0625, 0.25, 1, 0.0625, 0.0625, 0.25, 0.125]

exp_type = dir_seq
for fname in sorted(matches)[:3]:
    print fname
#     et = fname.split('\\')[-1].split('_')[1].split()[0]
#     rec = fname.split('\\')[-1].split('_')[2]
#     paradigm = fname.split('\\')[-1].split('_')[3]
    
    ls = []

    try:
        tmp =  pd.read_hdf(fname, key = 'raw') 
    except:
        continue
    trials =  int(tmp.shape[1]/1000)

    df3_array=np.reshape(tmp.values,(np.shape(tmp)[0],trials, -1))
    df3_avg=np.mean(df3_array,1)
    df3_avg = pd.DataFrame(df3_avg).T        
    df3_avg['times'] = times
    negativity_ch_idx=df3_avg[(df3_avg['times']>=0.35) & (df3_avg['times']< 0.45)].ix[:,8:16].min().idxmin()
#     negativity_ch_idx = negativity_ch_idx - 4
    print negativity_ch_idx
    # plotting the spectra and FFR for the maximum negativity channel, the first trial
    ddf = tmp.values[negativity_ch_idx,:]
    ddf2=np.reshape(ddf,(trials, 1000))
    ddf2 = pd.DataFrame(ddf2)
    ddf2['trial'] = exp_type[:ddf2.shape[0]]
    for j in ddf2.trial.unique():
    
        lfp = ddf2[ddf2.trial==j].iloc[:,:].mean().drop('trial')
#         if j==3 or j ==9:
#             plt.plot(lfp)
#             plt.show()
        tmp_df = pd.DataFrame({ 'lfp':lfp, 'paradigm': '12-dir', 'stim1':j, 'abs_times': times + j*1,
                               'fname':fname, 'times':times })
        ls.append(tmp_df)
    tmp_df2 = pd.concat(ls)
    ls2.append(tmp_df2)
    
df_tuning = pd.concat(ls2)
    

In [ ]:
ddf2.trial.unique()

In [ ]:
groups = {'205':1, '206':1, '207':1, '212':3, '213':3, '214':3 }
con_tuning['group'] = con_tuning.et.map(groups)

In [ ]:
idx = int(lfp_df.shape[0]/1000)*[np.arange(0,1,0.001)]
lfp_df['times']= np.concatenate(idx)


In [ ]:
sns.pal

## Load data

In [ ]:
path = r"U:\Data_Analysis\pak6\Analysis of LFP\SERT KO project\lfp_l4_sert_master.pkl"
df_sert = pd.read_pickle(path)

In [ ]:
# tmp = df_sert[df_sert.group == 'wt']
df_tmp = pd.concat([tmp, df_hdac]).reset_index(drop = True)

In [ ]:
df_master = sert_df

In [ ]:
df_sert_master = pd.concat([df_sert_da, df_sert_db])

In [ ]:
sert_df.groupby(['group', 'et', 'training']).fname.unique()

## Plotting avg traces

In [ ]:
# adjust time: stim1 at 1s. 380 pre, 378, 378 post
# stim1 at 0.5s: 009, 010 pre, rest at 1s
v = '380'
df_serta.loc[((df_serta.et == v) & (df_serta.training == 'pre')), 'times'] = df_serta.loc[((df_serta.et == v) 
                                                    & (df_serta.training == 'pre')), 'times'] - 0.50
df_serta = df_serta.round({'times': 3})

In [ ]:
df_master = pd.concat([df_sert, df_sert_l4])

In [ ]:
df_master.to_pickle('sert_l4_lfp_master.pkl')

In [ ]:
colors_training = ['gray', 'crimson']
colors_group = [ 'k', 'blue', 'magenta']

In [ ]:
for i,v in enumerate((sorted(df_sert_l4.et.unique()))[:]):
    df_sert_l4.loc[((df_sert_l4.et == v)), 'times'] = df_sert_l4.loc[((df_sert_l4.et == v)), 'times'] - 0.50
    df_sert_l4 = df_sert_l4.round({'times': 3})

In [ ]:
for i,v in enumerate((sorted(df_tmp[df_tmp.group == 'hdac'].et.unique()))):
    df_tmp.loc[((df_tmp.et == v) ), 'times'] = df_tmp.loc[((df_tmp.et == v)) , 'times'] - 0.50
#     df_tmp.loc[((df_tmp.et == v) ), 'times'] = df_tmp.loc[((df_tmp.et == v) ), 'times'].round({'times': 3})

In [ ]:
plt.figure(figsize=(10, 6), facecolor='w')
data = df_master
data = data[(data.stim1.str.contains('N') )
#               & (data.stim1 == 'G-1') 
            & (data.training == 'post')
               ] 
# data = data[(data.paradigm.str.contains('odd')) | (data.stim1 == 'ctr2')]
# data = data[(data.times > 0) & (data.times < 1.5) ]

colors_training = ['gray', 'crimson']
colors_group = [ 'k', 'magenta', 'blue']
# colors_group = ['k', 'coral']
# colors_stim = ['gray', 'crimson', 'green']
# data = data[(data.group != 'wat') & (data.layer == 'l4')]

# # data = data[data.rec.str.contains('1')]

data = data[(data.times > 0.05) & (data.times < 2.5)]
# data['abs_times'] = data['stim1'].str.split('ctr', expand = True)[1].astype('int')*0.3 + data['times']
# data = data[(data.stim1 == 1) | (data.stim1 == 3)]
# data = data[data.et == data.et.unique()[3]]
# print data.et.unique()
# data = data.groupby(['group' ,'stim1', 'rec']).mean().reset_index()
data = data.sort_values(by=['group', 'training'], ascending=[ False, False])

sns.lineplot( x = 'times',  y = 'lfp', data = data,
               legend = 'full', hue = 'group', 
             palette = colors_group,
           estimator = np.nanmean, ci = None
            )
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
# plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
sns.despine()
# plt.axhline(y = 0)
# plt.axvline(x = 0.5)
plt.axvspan(0.5, 0.9, facecolor="gray",alpha=0.2) 
# plt.xlim(0, 1)
plt.ylim(-350, 250)
# plt.savefig("line-sert-l4-gn-post.pdf", transparent=True)

In [ ]:
data.head()

In [ ]:
f, ax = plt.subplots( 1, 3, sharey = True, figsize =(12,6))
cbar_ax = f.add_axes([.91, .3, .03, .5])
vmin, vmax = -500, 500
data = lfp_df[(lfp_df.stim1.str.contains('ctr')) & (lfp_df.times > 0.3) 
              & (lfp_df.times < 0.6) & (lfp_df.layer == 'l2/3')]
tmp = data.groupby(['group', 'stim1', 'times']).lfp.mean().reset_index()
tmp2 = tmp[tmp.group == 'wt'].pivot('times', 'stim1', 'lfp')
tmp3 = tmp[tmp.group == 'fx'].pivot('times', 'stim1', 'lfp')
sns.heatmap(tmp2, cmap = 'jet', ax = ax[0], cbar = False, vmin = vmin, vmax = vmax)
sns.heatmap(tmp3, cmap = 'jet', ax = ax[1], cbar = False,  vmin = vmin, vmax = vmax)
sns.heatmap(tmp3.sub(tmp2), cmap = 'jet', ax = ax[2],  vmin = vmin, vmax = vmax, cbar_ax = cbar_ax)
ax[1].set_ylabel('')
ax[2].set_ylabel('')
ax[0].invert_yaxis()
# plt.savefig("hm-l23-sf_by_time.pdf", transparent=True)

In [ ]:
lfp_df.layer.unique()

In [ ]:
csd_df.head()

In [ ]:
Mean = csd_df.groupby(['fname', 'csd_step']).csd.transform('mean')    
Std = csd_df.groupby(['fname', 'csd_step']).csd.transform('std')
csd_df['zscore'] = (csd_df.csd.values - Mean) / Std

In [ ]:
tmp

In [ ]:
data = csd_df[(csd_df.paradigm.str.contains('odd') )
              | (csd_df.stim1.str.contains('ctr2'))
             ]
data = data[(data.times > 0.35) & (data.times < 0.7)]
data = data.groupby(['group', 'rec', 'layer' ,'stim1', 'csd_step']).min().reset_index()
tmp = data.groupby(['group', 'rec', 'layer' ,'stim1']).csd.mean().reset_index()
tmp = tmp.sort_values(by=['group', 'stim1'], ascending=[ False, False])
sns.factorplot(x = 'stim1', y = 'csd', data = tmp, palette = colors_group,
              hue = 'group', height = 5, aspect = 2,
                col = 'layer', col_wrap = 2, sharey = False,
              )
# plt.savefig("point-csd-across-layers.pdf", transparent=True)

In [ ]:
tmp = df_sert[(df_sert.stim1 == '1') & (df_sert.training == 'right1') 
              & (df_sert.times > 0) & (df_sert.times < 3)
             ]
for et in tmp.et.unique():
    print et
    tmp2 = tmp[tmp.et == et]
    plt.plot(tmp2.groupby('times').lfp.mean())
    plt.show()
#     break

In [ ]:
fig_inp = data[(data.times > 0.35) & (data.times < 0.5)]
colors = ['green', 'crimson', 'gray']
fig_inp = fig_inp.groupby([ 'rec', 'stim1']).min().reset_index()
fig_inp = fig_inp.sort_values(by = ['stim1', 'group'], ascending= [False, False])
g = sns.factorplot(x = "stim1", y = "lfp",   data = fig_inp,  kind = 'point', ci = 68, 
                   palette = colors_group, hue = 'group',
#                    capsize = 0.02, saturation = 1,
                   height = 5, aspect = 1.5 )
# plt.savefig("vep_bar_fx_mmn_l4.pdf", transparent=True)

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp

In [ ]:
fig_inp['lfp_tr'] = abs(fig_inp.lfp)**0.4
model = ols('lfp_tr ~ C(group)*C(stim1)', fig_inp).fit()

# Seeing if the overall model is significant
print("Overall model F", model.df_model,model.df_resid, model.fvalue, model.f_pvalue)

In [ ]:
# Durban-Watson: autocorrelation: 2(no), 0-2 (positive), 2-4 (negative)
model.summary()

In [ ]:
res = sm.stats.anova_lm(model, typ= 2)
res

In [ ]:
model2 = ols('lfp_tr ~ C(group)+ C(stim1)', fig_inp).fit()

# Seeing if the overall model is significant
print("Overall model F", model2.df_model,model2.df_resid, model2.fvalue, model2.f_pvalue)

In [ ]:
res2 = sm.stats.anova_lm(model2, typ= 2)
res2

In [ ]:
fig_inp['cond'] = fig_inp['stim1'] + fig_inp['group']

In [ ]:
fig_inp.groupby(['group', 'et']).stim1.unique()

In [ ]:
wt_std = fig_inp[(fig_inp.stim1 == 'std3') & (fig_inp.group == 'wt')].lfp.dropna().values
wt_dev = fig_inp[(fig_inp.stim1 == 'dev9') & (fig_inp.group == 'wt')].lfp.dropna().values
wt_ctr = fig_inp[(fig_inp.stim1 == 'ctr2') & (fig_inp.group == 'wt')].lfp.dropna().values

fx_std = fig_inp[(fig_inp.stim1 == 'std3') & (fig_inp.group == 'fx')].lfp.dropna().values
fx_dev = fig_inp[(fig_inp.stim1 == 'dev9') & (fig_inp.group == 'fx')].lfp.dropna().values
fx_ctr = fig_inp[(fig_inp.stim1 == 'ctr2') & (fig_inp.group == 'fx')].lfp.dropna().values

print np.mean(wt_std), sstat.sem(wt_std)
print np.mean(wt_dev), sstat.sem(wt_dev)
print np.mean(wt_ctr), sstat.sem(wt_ctr)


print (len(wt_std), len(fx_std))

# print sstat.kruskal(x1, x2, x3)

p1 =  sstat.mannwhitneyu(wt_std, wt_dev)[1]
p2 = sstat.mannwhitneyu(wt_std, wt_ctr)[1]
p3 =  sstat.mannwhitneyu(wt_dev, wt_ctr)[1]

p4 =  sstat.mannwhitneyu(fx_std, fx_dev)[1]
p5 =  sstat.mannwhitneyu(fx_std, fx_ctr)[1]
p6 =  sstat.mannwhitneyu(fx_dev, fx_ctr)[1]

p7 =  sstat.mannwhitneyu(wt_std, fx_std)[1]
p8 =  sstat.mannwhitneyu(wt_dev, fx_dev)[1]
p9 =  sstat.mannwhitneyu(wt_ctr, fx_ctr)[1]

pvals = np.array((p1, p2, p3, p4, p5, p6, p7, p8, p9))
adj = statsmodels.stats.multitest.multipletests(pvals, method='holm')
print pvals
print adj[1]

In [ ]:
fig_inp = lfp_df[(lfp_df.times > 0.4) & (lfp_df.times < 0.45)]
fig_inp = fig_inp[fig_inp.paradigm.str.contains('tun') & (fig_inp.layer == 'l4')]
fig_inp = fig_inp.groupby(['group', 'stim1', 'fname' ]).lfp.min().reset_index()
# fig_inp = fig_inp[(fig_inp.stim1.str.contains('ctr') )
#               & (fig_inp.layer == 'l5/6') 
#                 & (data.paradigm.str.contains('025')) 
#                 & (data.et == data.et.unique()[4])
#                ]   

fig_inp = fig_inp.sort_values(by = ['stim1', 'group'], ascending = [True, False])
g = sns.catplot(x="stim1", y="lfp",   data= fig_inp,  kind = 'point', ci = 68, 
                  hue = 'group', palette = ['k', 'coral'],
#                    capsize = 0.02, saturation = 1,
                   height = 5, aspect=1.5 )
# plt.ylim(-1500, 0)
plt.savefig('point-lfp-sf_tuning-04-045.pdf', transparent=True)

In [ ]:
val = 'lfp'
cond = 'stim1'

ls = []
# for lay in sorted(fig_inp['layer'].unique()):
#     print lay
for l in sorted(fig_inp[cond].unique()):

    stat_inp = fig_inp[fig_inp[cond] == l]
    x1 = stat_inp[(stat_inp.group == 'wt')   ][val].dropna().values
    x2 = stat_inp[(stat_inp.group == 'fx')   ][val].dropna().values

#     x3 = stat_inp[(stat_inp.group == 'wt') & (stat_inp.n_type == 'fs')][val].dropna().values
#     x4 = stat_inp[(stat_inp.group == 'fx') & (stat_inp.n_type == 'fs')][val].dropna().values

    print '\t', l
    wt_mean, wt_sem = np.mean(x1), sstat.sem(x1)
    fx_mean, fx_sem = np.mean(x2), sstat.sem(x2)
#     print 'wt, fs', len(x3) ,np.mean(x3), sstat.sem(x3)
#     print 'fx, fs', len(x4) ,np.mean(x4), sstat.sem(x4)

    x1_x2 = sstat.mannwhitneyu(x1, x2)

    data_tmp = pd.DataFrame({ 'stim1': l, 
'mean': [wt_mean, fx_mean], 'sem': [wt_sem, fx_sem], 
'n_ch': [len(x1), len(x2)], 'group': ['wt', 'fx'], 'p_val': [x1_x2[1], x1_x2[1]], 
            'u_stat': [x1_x2[0], x1_x2[0]]
                })
    ls.append(data_tmp)
    print '------------------------'
stat_table = pd.concat(ls)

In [ ]:
stat_table

In [ ]:
fig_inp['angle'] = fig_inp['stim1']*30
fig_inp['lfp'] = abs(fig_inp['lfp'])
tmp1 = fig_inp.groupby('angle').mean().reset_index()
std = fig_inp.groupby('angle').sem().reset_index()
tmp2 = tmp1.append(tmp1.iloc[0])
std2 = std.append(std.iloc[0])
ax = plt.subplot(111, projection='polar')
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)
ax.plot((tmp2.angle/360)*2*np.pi, abs(tmp2.lfp))
tmp2['a'] = tmp2.lfp + std2.lfp
tmp2['b'] = tmp2.lfp - std2.lfp
ax.fill_between((tmp2.angle/360)*2*np.pi, tmp2.a, tmp2.b, alpha=0.2)
ax.set_xticks(np.arange(0,2*np.pi,np.pi/6))


In [ ]:
path = r"U:\Data_Analysis\pak6\Analysis of LFP\FX mmn paper\fx_var-odd_lfp_11-22-18.pkl"
set1 = pd.read_pickle(path)
path = r"U:\Data_Analysis\pak6\Analysis of LFP\FX mmn paper\fx_var-odd_lfp_12-18-18.pkl"
set2 = pd.read_pickle(path)
path = r"U:\Data_Analysis\pak6\Analysis of LFP\FX mmn paper\thy1-fx_var-odd_lfp.pkl"
set3 = pd.read_pickle(path)

In [ ]:
dir_seq.size/12

In [ ]:
mmn_df = pd.concat([set1, set2, set3]).reset_index(drop = True)

In [ ]:
x = np.arange(0, 1, 0.001)
for fname in sorted(mmn_df3.fname.unique()[160:]):
    tmp = mmn_df3[(mmn_df3.fname == fname)]
    try:
        print tmp.et.values[0], tmp.group.values[0], tmp.side.values[0] , tmp.paradigm.values[0],tmp.l4_ch.values[0]
    except:
        print 'no'
    ctr = tmp[tmp.stim1 == 'ctr2'].lfp.values
    std = tmp[(tmp.stim1 == 'std3') & (tmp.paradigm == 'sf-nov-odd')].lfp.values
    dev = tmp[tmp.stim1 == 'dev9'].lfp.values
    if ctr.shape[0] > 0:
        plt.plot(x, ctr, c = 'magenta')
    if std.shape[0] > 1:
        plt.plot(x, std, c = 'gray')
        plt.plot(x, dev, c = 'r')
    plt.axvline(x = 0.4)
    plt.show()
    
    
    

In [ ]:
np.diff(tmp.T.dropna().T.iloc[0].index.values)*1e3

In [ ]:
# f, ax = plt.subplots()
tmp = data[data.stim1 == 'std3'].dropna().pivot(index = 'fname', columns= 'times' ,values = 'lfp') 
# tf, time, frex, tf3d = oem.tf_cmw(ax, tmp, show=True, log_scale=False)

In [ ]:
colors_training = ['gray', 'crimson']
colors_group = [ 'k', 'blue', 'magenta']

In [ ]:
# data = mmn_exp[mmn_exp.paradigm =='paired-oddball']
data = df_master
data = data[(data.stim1.str.contains('N'))
#               & (data.layer == 'l4') 
            & (data.training == 'post')
               ] 

vep1 = data[(data.times>0.53) & (data.times<0.63)]
vep1 = vep1.ix[vep1.groupby([ 'stim1' ,'et']).lfp.idxmin().values]

vep1.loc[:, 'vep'] = 1

vep2 = data[(data.times > 0.73) & (data.times < 0.83)]
vep2 = vep2.ix[vep2.groupby('et').lfp.idxmin().values]

vep2.loc[:, 'vep'] = 2

vep3 = data[(data.times > 0.93) & (data.times < 1.03)]
vep3 = vep3.ix[vep3.groupby('et').lfp.idxmin().values]

vep3.loc[:, 'vep'] = 3

out = pd.concat([vep1, vep2, vep3])
# _data = band_df
g = sns.factorplot(x="vep", y="lfp", hue ='group',  data=out,  kind = 'bar', ci = 68, 
                   palette = colors_group, hue_order = ['wt', 'het', 'ko'],
                   capsize = 0.02, saturation = 1,
                   size = 5, aspect=1.5 )

# plt.savefig('bar-sert-g1-pre-l4.pdf', transparent=True)

In [ ]:
tf_out3.head()

In [ ]:
cond = 'variable'
val = 'value'
_inp = tf_out3
for l in sorted(_inp[cond].unique()):
    stat_inp = _inp[_inp[cond] == l]
#     stat_inp = stat_inp.groupby(['cond', 'et']).mean().reset_index()
    x1 = stat_inp[(stat_inp.cond == 'wt')   ][val].dropna().values
    x2 = stat_inp[(stat_inp.cond == 'het')   ][val].dropna().values
    x3 = stat_inp[(stat_inp.cond == 'ko')   ][val].dropna().values

    print '\t', l
    print 'wt mean sem', np.mean(x1), sstat.sem(x1)
    print 'het mean sem', np.mean(x2), sstat.sem(x2)
    print 'ko mean sem', np.mean(x3), sstat.sem(x3)
    print len(x1), len(x2), len(x3)
    
    print sstat.kruskal(x1, x2, x3)
    print sstat.mannwhitneyu(x1, x2)
    print sstat.mannwhitneyu(x1, x3)
    print sstat.mannwhitneyu(x2, x3)
    print '------------------------'

In [ ]:
#test = master_df[master_df.training=='pre'].reset_index()
test = data

vep1 = test[(test.times>0.37) & (test.times<0.47)]
vep1 = vep1.ix[vep1.groupby([ 'stim1' ,'fname']).lfp.idxmin().values]
vep1 = vep1.groupby([ 'stim1' ,'fname']).min()
vep1 = vep1.reset_index()
vep1['vep'] = 'ssa'

vep2 = test[(test.times>0.6) & (test.times<0.7)]
vep2 = vep2.ix[vep2.groupby(['stim1' ,'fname']).lfp.idxmax().values]
vep2 = vep2.groupby( ['stim1', 'fname']).max()
vep2 = vep2.reset_index()
vep2['vep'] = 'dd'

# vep3 = test[(test.times>0.93) & (test.times<1.03)]
# vep3 = vep3.ix[vep3.groupby('fname').lfp.idxmin().values]
# vep3 = vep3.groupby('fname').min()
# vep3['vep'] = 3

# vep4 = test[(test.times>1.15) & (test.times<1.25)]
# vep4 = vep4.ix[vep4.groupby('fname').lfp.idxmin().values]
# vep4 = vep4.groupby('fname').min()
# vep4['vep'] = 4

# vep5 = test[(test.times>1.38) & (test.times<1.48)]
# vep5 = vep5.ix[vep5.groupby('fname').lfp.idxmin().values]
# vep5 = vep5.groupby('fname').min()
# vep5['vep'] = 5



out = pd.concat([vep1, vep2])
# out = out.sort_values(by=['group'])
# _data = band_df
g = sns.factorplot(x="vep", y="lfp", hue ='stim1',  data=out,  kind = 'bar', ci = 95, 
#                     capsize = 0.02, saturation = 1,
                   size = 5, aspect=1.5 )
# plt.ylim(-700, 700)
# plt.savefig('sf-vmmn-l4vep_quant-groupb.pdf', transparent=True)
# plt.savefig('sert_ko_amp_vep_all.png', transparent=True)


In [ ]:
print sstat.wilcoxon(vep1[vep1.stim1 == 3].lfp, vep1[vep1.stim1 == 9].lfp )
print sstat.wilcoxon(vep2[vep2.stim1 == 3].lfp, vep2[vep2.stim1 == 9].lfp )


In [ ]:
df_hdac3.to_pickle('lfp_l4_hdac.pkl')

In [ ]:
path = r"U:\Data_Analysis\pak6\Analysis of LFP\fx-mmn-paper-tf_power.pkl"
tf_df = pd.read_pickle(path)

In [ ]:
tf_df.head()

## TF analysis

In [ ]:
data = mmn_df
data = data[(data.paradigm.str.contains('odd') )
              | (data.stim1.str.contains('ctr2')) 
#                 | (data.stim1.str.contains('std')) 
#                 & (data.et == data.et.unique()[4])
               ]   

data = data[(data.group == 'fx') & (data.layer == 'l4')]
data = data[(data.times>0) & (data.times<0.95)]

In [ ]:
_data

In [ ]:
ls = []
ls2= []
ls_perm = []
sf_col = [0.01, 0.02, 0.04, 0.08, 0.14 ]
data = df_master
data = data[(data.stim1.str.contains('1'))
#               & (data.layer == 'l4') 
            & (data.training == 'post')
               ] 
# data = data[(data.paradigm.str.contains('odd')) | (data.stim1 == 'ctr2') ]
data = data[(data.times > 0.05) & (data.times < 2.5)]
data = data.groupby(['group' ,'et', 'times']).mean().reset_index()
data = data.sort_values(by=['group'], ascending=[False])
_data = data
# _data = _data[_data.cnt == '25']
param = 'group'
n =sorted(_data[param].unique()) 
f, ax = plt.subplots( 1, len(n), sharex=True, sharey=True, figsize=(12,6), facecolor = 'w')
t1, t2 = 500, 1500

for idx, val in enumerate((_data[param].unique())):
    
    #tmp = _data[_data['stim1']==sf].pivot_table(index = ['fname', 'et'], columns= 'times' ,values = 'lfp').mean(level=1)
    tmp = _data[_data[param]==val].pivot(index = 'et', columns= 'times' ,values = 'lfp') 
    tmp = tmp.T.dropna().T
    tf, time, frex, tf3d = oem.tf_cmw(ax[idx], tmp, show=True, log_scale=False)
    ax[idx].set_title('' + str(val))
    
    tf3_arr = np.dstack(tf3d)
    tf3_arr = 10*np.log10(tf3_arr/tf3_arr[:,:500,:].mean(axis=1, keepdims=1))
    
#     mask = frex[(frex>12) & (frex<25)]
#     ax[idx].plot(((tf[1, int(mask[0]):int(mask[-1]), :]).mean(axis=0)))
    ls_perm.append(tf3_arr)
    # this is because dB averaging does not equal to linear power averaging 
    tmp = np.dstack(tf3d)
    tmp = tmp.swapaxes(0,2)
    tmp = tmp/tmp[:,:500,:].mean(axis=1, keepdims=1)
    #include only positve power, quanitfy bands
   
    alpha = np.mean(tmp[(frex>8) & (frex<=12)][:,t1:t2,:], axis=1)
    alpha = np.mean(alpha,axis=0)

    beta = np.mean(tmp[(frex > 12) & (frex <= 30)][:,t1:t2,:], axis=1)
    beta = np.mean(beta, axis=0)

    low_gamma = np.mean(tmp[(frex > 30) & (frex <= 50)][:,t1:t2,:], axis=1)
    low_gamma = np.mean(low_gamma,axis=0)

    high_gamma = np.mean(tmp[(frex>50)][:,t1:t2,:] , axis=1)
    high_gamma = np.mean(high_gamma,axis=0)

    theta = np.mean(tmp[(frex>=4) & (frex<=8)][:,t1:t2,:], axis=1)
    theta = np.mean(theta,axis=0)

    tf_tmp = pd.DataFrame({'cond': val,  'beta':beta, 'hg': high_gamma,
                           'alpha': alpha,   'lg': low_gamma, 'theta': theta,
                   'index': np.arange(0, _data[_data[param]==val].et.unique().size)
                     })
    ls2.append(tf_tmp)
tf_tmp = pd.concat(ls2)
# tf_tmp = pd.melt(tf_out, id_vars=['cond'], value_vars=['theta', 'alpha', 'beta', 'lg','hg' ])
tf_tmp = pd.melt(tf_tmp, id_vars=['cond','index' ], 
                 value_vars=['theta', 'alpha', 'beta',  'lg','hg' ])
tf_out3 = tf_tmp.groupby(['cond','variable', 'index']).mean().reset_index()
plt.tight_layout()


# plt.savefig("nat-mov_tf.png", transparent=True)
# plt.savefig("cbar.png", transparent=True)

In [ ]:
tf_out3['ucond'] = 'postn'


In [ ]:
tmp = np.dstack(tf3d)
tmp = tmp.swapaxes(0,2)
tmp = tmp/tmp[:,:300,:].mean(axis=1, keepdims=1)
#include only positve power, quanitfy bands


In [ ]:
tf_out3.head()


In [ ]:
ls = []
ls2= []
ls_perm = []
sf_col = [0.01, 0.02, 0.04, 0.08, 0.14 ]
# _data = sert_df[(sert_df.training == 'post')
#               & (sert_df.stim1 == '1')
#               & (sert_df.stim2 == 'G')
#                & (sert_df.times < 2.5)]
_data = df_sf3
param = 'training'
n =sorted(_data[param].unique()) 
f, ax = plt.subplots( 1, len(n), sharex=True, sharey=True, figsize=(14,4), facecolor = 'w')
t1, t2 = 400, 500
ls_t2 = [120, 190, 260, 330, 400, 470, 540]
#ls_high = []
# _data = _data[  (_data.training_type == 'ori') 
#                 & (_data.training == 'oddball') 
#                & (_data.stim1==3)
#                 & (_data.et != 'KIC5')
#                ]  
# _data = tf_exp
# times = np.linspace(0,0.5,500)


for idx, val in enumerate(sorted(_data[param].unique(), reverse=True)):
    
    #tmp = _data[_data['stim1']==sf].pivot_table(index = ['fname', 'et'], columns= 'times' ,values = 'lfp').mean(level=1)
    tmp = _data[_data[param]==val].pivot(index = 'fname', columns= 'times' ,values = 'lfp') 
 
    tf, time, frex, tf3d = oem.tf_cmw(ax[idx], tmp, show=False, log_scale=False)
    ax[idx].set_title('' + str(val))
    
#     tf3_arr = np.dstack(tf3d)
#     tf3_arr = 10*np.log10(tf3_arr/tf3_arr[:,:500,:].mean(axis=1, keepdims=1))
    
#     mask = frex[(frex>12) & (frex<25)]
#     ax[idx].plot(((tf[1, int(mask[0]):int(mask[-1]), :]).mean(axis=0)))
#     ls_perm.append(tf3_arr)
    
    tmp = np.dstack(tf3d)
    tmp = tmp.swapaxes(0,2)
    tmp = 10*np.log10(tmp/tmp[:,:500,:].mean(axis=1, keepdims=1))
    for t2 in ls_t2:
        t1 = t2-70

        alpha = np.max(tmp[(frex>8) & (frex<=12)], axis=0)[500+t1: 500+t2]
        alpha = np.mean(alpha,axis=0)

        beta = np.max(tmp[(frex>12) & (frex<=25)], axis=0)[500+t1: 500+t2]
        beta = np.mean(beta,axis=0)

        low_gamma = np.max(tmp[(frex>30) & (frex<=50)], axis=0)[500+t1: 500+t2]
        low_gamma = np.mean(low_gamma,axis=0)

        high_gamma = np.max(tmp[(frex>50) ], axis=0)[500+t1: 500+t2]
        high_gamma = np.mean(high_gamma,axis=0)

#         theta = np.mean(tmp[(frex>=4) & (frex<=8)], axis=0)[1000*tw+50+t1: 1000*tw+50+t2]
#         theta = np.mean(theta,axis=0)


        tf_tmp = pd.DataFrame({'cond': val,  'beta':beta, 'hg': high_gamma,'window':str(t2/1000),
                               'alpha': alpha,   'lg': low_gamma, 
                       'index': np.arange(0, _data[_data[param]==val].fname.unique().size)
                 })
        ls.append(tf_tmp)
    tf_tmp = pd.concat(ls)
    ls2.append(tf_tmp)
tf_out = pd.concat(ls2)
tf_tmp = pd.melt(tf_out, id_vars=['cond'], value_vars=['alpha', 'beta', 'lg','hg' ])
tf_out = pd.melt(tf_out, id_vars=['window' ,'cond','index' ], 
                 value_vars=['alpha', 'beta',  'lg','hg' ])
tf_out = tf_out.groupby(['window' ,'cond','variable', 'index']).mean().reset_index()
plt.tight_layout()
# plt.savefig("nat-mov_tf.png", transparent=True)
# plt.savefig("nat-mov_tf.pdf", transparent=True)

In [ ]:
tf_out4 = pd.concat([ tf_out, tf_out2, tf_out3])

In [ ]:
tf_out.groupby(['cond', 'variable']).mean()

In [ ]:
colors = ['green', 'crimson', 'gray']
fig_inp = tf_out4
# remove outliers
# fig_inp = fig_inp[fig_inp.groupby(['variable', 'group']).value.
#       transform(lambda x : (x<x.quantile(0.95))&(x>(x.quantile(0.05)))).eq(1)]
fig_inp = fig_inp.sort_values(by = ['cond'], ascending = [False])
g = sns.catplot(x="ucond", y="value", hue = 'cond',  data=fig_inp,  kind = 'bar', order = ['1pre', '1post', 'postn'],
                    ci = 68,  sharey = False, legend = True, hue_order = ['wt', 'het', 'ko'],
#                 order = reversed(['theta', 'alpha', 'beta', 'lg', 'hg']),
                   palette = colors_group, 
                col = 'variable', col_wrap = 2,
#                     capsize = 0.02, saturation = 1, col_wrap = 2,
                   height = 4, aspect = 1.2 )
plt.savefig("bar_tf-sert-l4-05-15.pdf", transparent=True)

In [ ]:
tf_out2

In [ ]:
fig_inp.head()

In [ ]:
tmp = fig_inp[fig_inp.variable == 'hg']

wt_std = tmp[(tmp.cond == 'std3') & (tmp.group == 'wt')].value.dropna().values
wt_dev = tmp[(tmp.cond == 'dev9') & (tmp.group == 'wt')].value.dropna().values
wt_ctr = tmp[(tmp.cond == 'ctr2') & (tmp.group == 'wt')].value.dropna().values

fx_std = tmp[(tmp.cond == 'std3') & (tmp.group == 'fx')].value.dropna().values
fx_dev = tmp[(tmp.cond == 'dev9') & (tmp.group == 'fx')].value.dropna().values
fx_ctr = tmp[(tmp.cond == 'ctr2') & (tmp.group == 'fx')].value.dropna().values

print np.mean(wt_std), sstat.sem(wt_std)
print np.mean(wt_dev), sstat.sem(wt_dev)
print np.mean(wt_ctr), sstat.sem(wt_ctr)


print (len(wt_std), len(fx_std))

# print sstat.kruskal(x1, x2, x3)

p1 =  sstat.mannwhitneyu(wt_std, wt_dev)[1]
p2 = sstat.mannwhitneyu(wt_std, wt_ctr)[1]
p3 =  sstat.mannwhitneyu(wt_dev, wt_ctr)[1]

p4 =  sstat.mannwhitneyu(fx_std, fx_dev)[1]
p5 =  sstat.mannwhitneyu(fx_std, fx_ctr)[1]
p6 =  sstat.mannwhitneyu(fx_dev, fx_ctr)[1]

p7 =  sstat.mannwhitneyu(wt_std, fx_std)[1]
p8 =  sstat.mannwhitneyu(wt_dev, fx_dev)[1]
p9 =  sstat.mannwhitneyu(wt_ctr, fx_ctr)[1]

pvals = np.array((p1, p2, p3, p4, p5, p6, p7, p8, p9))
adj = statsmodels.stats.multitest.multipletests(pvals, method='fdr_bh')
print pvals
print adj[1]

In [ ]:
ls_band = []
bands = ['theta', 'alpha', 'beta', 'lg', 'hg']
for idx, var in enumerate(bands):
    
    tmp = fig_inp[fig_inp.variable == var]
    ls_layer = []
    for lidx, lval in enumerate(sorted(tmp.layer.unique())):
        tmp2 = tmp[tmp.layer == lval]
        std = tmp2[tmp2.cond == 'std3'].value.dropna().values
        dev = tmp2[tmp2.cond == 'dev9'].value.dropna().values
        ctr = tmp2[tmp2.cond == 'ctr2'].value.dropna().values
        print var, lval
        std_mean, std_sem = np.mean(std), sstat.sem(std)
        dev_mean, dev_sem = np.mean(dev), sstat.sem(dev)
        ctr_mean, ctr_sem = np.mean(ctr), sstat.sem(ctr)
        
        
        print len(std), len(dev), len(ctr)
#         print sstat.kruskal(std, dev, ctr)

        std_dev = sstat.mannwhitneyu(std, dev)
        std_ctr = sstat.mannwhitneyu(std, ctr)
        dev_ctr = sstat.mannwhitneyu(dev, ctr)

        data_tmp = pd.DataFrame({'layer': lval, 'band': var, 'stim1': ['std', 'dev', 'ctr'], 
        'mean': [std_mean, dev_mean, ctr_mean ], 'sem': [std_sem, dev_sem, ctr_sem ], 
        'n_ch': [len(std), len(dev), len(ctr)], 'group': 'wt', 'comparison': ['std_dev', 'std_ctr', 'dev_ctr'],
        'p_val': [std_dev[1], std_ctr[1], dev_ctr[1]], 'u_stat': [std_dev[0], std_ctr[0], dev_ctr[0]],
                                 'test': 'MWU' })
        ls_layer.append(data_tmp)
        
        print "--------------------------"
    print '========================'
    ls_band.append(pd.concat(ls_layer))
stat_table = pd.concat(ls_band)
stat_table.head()

In [ ]:
cols = ['group', 'band', 'layer', 'comparison', 'test', 'u_stat', 'p_val', 'stim1', 'n_ch', 'mean', 'sem' ]
stat_table[cols]

In [ ]:
tmp = tf_df[tf_df.groupby(['variable', 'group']).value.
      transform(lambda x : (x<x.quantile(0.95))&(x>(x.quantile(0.05)))).eq(1)]

## CSD Analysis

In [ ]:
tmp.groupby(['variable', 'group'])["value"].quantile([0.05, 0.95]).unstack(level=1)

In [ ]:
tmp = np.dstack(tf3d)
tmp = tmp.swapaxes(0,2)

In [ ]:
frex

In [ ]:
frex

### Permutation test

In [ ]:
# tf3 = np.array_split(tf3d.as_matrix(), 108)
# make 3d array from list of 2d arrays

tf3_arr = np.dstack(tf3d)
tf3_arr = tf3_arr.swapaxes(0,2)
tf3_pre = 10*np.log10(tf3_arr/tf3_arr[:,:400,:].mean(axis=1, keepdims=1))
# reshape to 2d array for dataframe
f,s,tr = tf3_pre.shape
tf3d_pre = pd.DataFrame(tf3_pre.transpose(2,0,1).reshape(f*tr,-1))

In [ ]:
f, ax = plt.subplots(5,1, sharex=True, sharey=True, figsize=(8,12))
# time = np.linspace(0, 4, 4000 )
diff_map = post_map-pre_map
sf_col = [0.01, 0.02, 0.04, 0.08, 0.14 ]
contour_levels = np.arange(-10, 30, 1)
for i, val in enumerate(sf_col):
    ax[i].set_yscale('log')
    ax[i].set_yticks(np.logspace(np.log10(2),np.log10(80),6))
    ax[i].set_yticklabels(np.round(np.logspace(np.log10(2),np.log10(80),6)))
    
    hm = ax[i].contourf(time, frex, post_map[i], 40,  cmap = 'jet' , extend = 'both' )
    ax[i].set_ylabel('Frequency')
    cb_tf = plt.colorbar(hm, ax=ax[i])
    cb_tf.set_label('dB from baseline')
    ax[i].set_title(val)

In [ ]:

T_obs, clusters, cluster_p_values, H0 = mne.stats.permutation_cluster_test([ls_perm[1], ls_perm[0]],
                             n_permutations=256, threshold = 15.0,  tail=0)
T_obs_plot = np.nan * np.ones_like(T_obs)
for c, p_val in zip(clusters, cluster_p_values):
    if p_val <= 0.05:
        T_obs_plot[c] = T_obs[c]

f, ax = plt.subplots(figsize = (5,5))
# ax.set_xlim(0,2.5)
# ax.set_yscale('log')
# hm = ax.set_yticks(np.logspace(np.log10(2),np.log10(80),6))
# ax.set_yticklabels(np.round(np.logspace(np.log10(2),np.log10(80),6)))
# ax.contourf(time, frex, T_obs.T, 
           
#            aspect='auto', origin='lower', cmap='gray')
hm = ax.contourf(time, frex, T_obs_plot.T, 
           
           aspect='auto', origin='lower', cmap='jet')
# plt.colorbar(hm)
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
# plt.savefig("tf_permutation.png", transparent=True)
# plt.savefig("tf_permutation_sf_low2.pdf", transparent=True)

In [ ]:
path = r"U:\Papers\Impaired Visual Familiarity Circuit in Fmr1 KO Mice\Sam\Data analysis for figures\RAW\Phase Shift\wtfx_all.hdf5"
df_units = pd.read_hdf(path)

In [ ]:
df_fx.head()

In [ ]:
a1

In [ ]:
import neurodsp
x = lfp
Fs = 1000
all_pac_methods = ['ozkurt', 'plv', 'glm', 'tort', 'canolty']
f_range_lo = (13, 30)
f_range_hi = (50, 200)
N_seconds_lo = .25
N_seconds_hi = .2

np.random.seed(0) # The tort and canolty methods uses random methods
pacs = np.zeros(len(all_pac_methods))
for i, m in enumerate(all_pac_methods):
    pacs[i] = neurodsp.compute_pac(x, x, Fs, f_range_lo, f_range_hi,
                               N_seconds_lo=N_seconds_lo, N_seconds_hi=N_seconds_hi,
                               pac_method=m)
    print('PAC strength = {:.4f}, {:s} method'.format(pacs[i], m))



In [ ]:

# Parameters for comodulogram function
f_pha_bin_edges = np.arange(3, 22, 2)
f_amp_bin_edges = np.arange(20, 120, 4)
N_cycles_pha = 3
N_cycles_amp = 11

# Compute comodulogram
pac = neurodsp.compute_pac_comodulogram(x, x, Fs,
                                        f_pha_bin_edges, f_amp_bin_edges,
                                        N_cycles_pha=N_cycles_pha,
                                        N_cycles_amp=N_cycles_amp,
                                        pac_method='ozkurt')



In [ ]:
# Plot comodulogram
neurodsp.plot_pac_comodulogram(pac, f_pha_bin_edges, f_amp_bin_edges,
                               clim=(0,.1))

## SF decoding from LFP data

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score, StratifiedKFold
from collections import defaultdict
from statsmodels.formula.api import ols
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC

In [ ]:
master_df['group'] = master_df.et.map(groups)
master_df['trial_n'] = master_df.index//1e3
master_df['rect_LFP'] = abs(master_df.lfp)

In [ ]:
master_df.head()

In [ ]:
# clf = svm.SVC(kernel='linear', C=1)
ls_cond = []
rskf = RepeatedStratifiedKFold(n_splits = 4, n_repeats = 5,
     random_state = 3)
for gr in df2.group.unique():
    for w in time_w:
        df2_sub = df2[(df2.trial_spikes >= w[0]) & (df2.trial_spikes < w[1]) & (df2.group == gr)]
        resp_units = df2_sub.groupby('cluster_id').times.count().reset_index()
        resp_units = resp_units[resp_units.times > 10].cluster_id.unique()
        tmp = df2_sub[df2_sub.cluster_id.isin(resp_units)].groupby(['cluster_id', 
                                                'trial_n']).trial_spikes.count().reset_index()
        tmp2 = tmp.pivot('trial_n', 'cluster_id', 'trial_spikes')
        tmp2 = tmp2.fillna(0)

        ls = []
# coef_arr = np.zeros((100,5, _data.cuid.unique().size ))
        print gr, w

        x_ = tmp2.values
        y_ = sf_tuning.values()

        scores = cross_val_score(lr, x_, y_, cv = rskf)
#         print scores
        tmp_df = pd.DataFrame({'cv': i, 'group': gr, 'window': w[0], 'acc': scores})
        ls_cond.append(tmp_df)

In [ ]:
rskf = RepeatedStratifiedKFold(n_splits = 4, n_repeats = 5,
     random_state = 3) 
lda = LinearDiscriminantAnalysis(solver = 'lsqr', shrinkage = 'auto')
# svm = SVC(C=1)
y_ = sf_tuning
time_w = [(0.35, 0.8), (0.35, 0.45), (0.45, 0.55), (0.55, 0.65), (0.65, 0.75)]
ls_cond = []
for gr in master_df.group.unique():
    for w in time_w:
        
        df_sub = master_df[(master_df.group == gr) & (master_df.times > w[0]) & (master_df.times < w[1])]
#         df_sub = df_sub.groupby(['group', 'rec', 'trial_n', 'times']).mean().reset_index()
        df_sub2 = df_sub.groupby(['fname', 'trial_n']).rect_LFP.mean().reset_index()
        x_ = df_sub2.pivot('trial_n', 'fname', 'rect_LFP')
#         X_scaled = preprocessing.scale(x_)
        x_ -= x_.min() 
        x_ /= x_.max()
        scores = cross_val_score(lda, x_, y_, cv = rskf)
        
        tmp_df = pd.DataFrame({'cv': i, 'group': gr, 'window': w[0], 'acc': scores})
        ls_cond.append(tmp_df)
out = pd.concat(ls_cond)
out.loc[:,'error'] = 1 - out.acc
out.head()

In [ ]:
sns.catplot(x = 'window', y = 'error', data = out, hue = 'group', 
            kind = 'point', height = 6, aspect = 1.5, palette = colors_group , ci = 95)
plt.ylim(0, 0.8)

In [ ]:
df_sub.head()

In [ ]:
x_.shape